In [1]:
from numba import jit
import numpy as np
from jampy.quadva import quadva
import astropy.units as u

## jit

In [24]:
@jit(nopython=True) # Set "nopython" mode for best performance, equivalent to @njit
def alpha_x(tau1, 
            y, x, M, sigma, q):

    tau = tau1.reshape(tau1.size, 1)
   
    x_til = x/sigma
    y_til = y/sigma

   
    
    eta = np.sqrt(1 - q**2)
    eta_sq = 1 - q**2
    
    aux = (M/sigma)*(x_til/(np.sqrt(1 - eta_sq*(tau)**2)))
    exp_arg = (tau**2/2)*(x_til**2 + y_til**2/(1-eta_sq*(tau)**2))
    exp = np.exp(-exp_arg)
    

    
    arr = aux*exp
    return tau1*np.sum(arr, 1)

In [3]:
@jit(nopython=True) # Set "nopython" mode for best performance, equivalent to @njit
def alpha_y(tau1,
            y, x, M, sigma, q):
    

    tau = tau1.reshape(tau1.size, 1)
    x_til = x/sigma
    y_til = y/sigma
    eta = np.sqrt(1 - q**2)
    eta_sq = 1 - q**2
    
    aux = (M/sigma)*(y_til/(np.power(1 - eta_sq*(tau)**2, 3/2)))
    exp_arg = (tau**2/2)*(x_til**2 + y_til**2/(1-eta_sq*(tau)**2))
    exp = np.exp(-exp_arg)
    
    
    arr = aux*exp
    
    return tau1*np.sum(arr, 1)

In [4]:
M, sigma, q = np.loadtxt("/home/carlos/Downloads/Integration Tests MGE/Input.txt", unpack=True)
grid = np.loadtxt("/home/carlos/Downloads/Integration Tests MGE/grid.txt")
grid = (grid*u.arcsec).to(u.rad).value

In [5]:
result_x = np.zeros([len(grid), 3])              #Onde ficarão salvos os resultados da deflexão em x
result_y = np.zeros([len(grid), 3])              #Onde ficarão salvos os resultados da deflexão em y

Time for one integral

In [7]:
%timeit quadva(alpha_x, [0., 1.], args=(grid[0][0], grid[0][1], M, sigma, q))   #Integral em x
%timeit quadva(alpha_y, [0., 1.], args=(grid[0][0], grid[0][1], M, sigma, q))   #Integral em y

810 µs ± 4.63 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.05 ms ± 8.22 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [32]:
start = time.time()
quadva(alpha_x, [0., 1.], args=(grid[0][0], grid[0][1], M, sigma, q))   #Integral em x
print("alpha_x elapsed time:", time.time() - start)

alpha_x elapsed time: 0.001712799072265625


Time for all integrais

In [11]:
%%timeit
for i in range(len(grid)):                      #Começo do loop
    result_x[i] = quadva(alpha_x, [0., 1.], args=(grid[i][0], grid[i][1], M, sigma, q))   #Integral em x
    result_y[i] = quadva(alpha_y, [0., 1.], args=(grid[i][0], grid[i][1], M, sigma, q))   #Integral em y


24.7 s ± 218 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## njit

In [6]:
def test_x(tau1, 
            y, x, M, sigma, q):
    tau = np.reshape(tau1.copy(), (tau1.size,1))

   
    x_til = x/sigma
    y_til = y/sigma

   
    
    eta = np.sqrt(1 - q**2)
    eta_sq = 1 - q**2
    
    aux = (M/sigma)*(x_til/(np.sqrt(1 - eta_sq*(tau)**2)))
    exp_arg = (tau**2/2)*(x_til**2 + y_til**2/(1-eta_sq*(tau)**2))
    exp = np.exp(-exp_arg)
    

    
    arr = aux*exp    
    return tau1*np.sum(arr, 1)

In [7]:
def test_y(tau1,
            y, x, M, sigma, q):
    tau = np.reshape(tau1.copy(), (tau1.size,1))
    
    x_til = x/sigma
    y_til = y/sigma
    eta = np.sqrt(1 - q**2)
    eta_sq = 1 - q**2
    
    aux = (M/sigma)*(y_til/(np.power(1 - eta_sq*(tau)**2, 3/2)))
    exp_arg = (tau**2/2)*(x_til**2 + y_til**2/(1-eta_sq*(tau)**2))
    exp = np.exp(-exp_arg)
    
    
    arr = aux*exp
    
    return tau1*np.sum(arr, 1)

In [8]:
from numba import njit, double

In [9]:
test_x_nb = njit(double[:](double[:],double,double,double[:],double[:],double[:]))(test_x)
test_y_nb = njit(double[:](double[:],double,double,double[:],double[:],double[:]))(test_y)

Time for one integral

In [18]:
%timeit quadva(test_x_nb, [0., 1.], args=((grid[0][0], grid[0][1], M, sigma, q)))   #Integral em x
%timeit quadva(test_y_nb, [0., 1.], args=((grid[0][0], grid[0][1], M, sigma, q)))   #Integral em y

834 µs ± 20 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.07 ms ± 12 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Time for all integrals

In [19]:
%%timeit
for i in range(len(grid)):                      #Começo do loop
    result_x[i] = quadva(test_x_nb, [0., 1.], args=(grid[i][0], grid[i][1], M, sigma, q))   #Integral em x
    result_y[i] = quadva(test_y_nb, [0., 1.], args=(grid[i][0], grid[i][1], M, sigma, q))   #Integral em y


24.7 s ± 69.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## nojit

In [20]:
def no_alpha_x(tau1, 
            y, x, M, sigma, q):

    tau = tau1[:, None]
   
    x_til = x/sigma
    y_til = y/sigma

   
    
    eta = np.sqrt(1 - q**2)
    eta_sq = 1 - q**2
    
    aux = (M/sigma)*(x_til/(np.sqrt(1 - eta_sq*(tau)**2)))
    exp_arg = (tau**2/2)*(x_til**2 + y_til**2/(1-eta_sq*(tau)**2))
    exp = np.exp(-exp_arg)
    

    
    arr = aux*exp
    return tau1*np.sum(arr, 1)

In [21]:
def no_alpha_y(tau1,
            y, x, M, sigma, q):
    

    tau = tau1[:, None]
    x_til = x/sigma
    y_til = y/sigma
    eta = np.sqrt(1 - q**2)
    eta_sq = 1 - q**2
    
    aux = (M/sigma)*(y_til/(np.power(1 - eta_sq*(tau)**2, 3/2)))
    exp_arg = (tau**2/2)*(x_til**2 + y_til**2/(1-eta_sq*(tau)**2))
    exp = np.exp(-exp_arg)
    
    
    arr = aux*exp
    
    return tau1*np.sum(arr, 1)

Time for one integral

In [22]:
%timeit quadva(no_alpha_x, [0., 1.], args=((grid[0][0], grid[0][1], M, sigma, q)))   #Integral em x
%timeit quadva(no_alpha_y, [0., 1.], args=((grid[0][0], grid[0][1], M, sigma, q)))   #Integral em y

879 µs ± 6.24 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.14 ms ± 8.19 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Time for all integrals

In [23]:
%%timeit
for i in range(len(grid)):                      #Começo do loop
    result_x[i] = quadva(no_alpha_x, [0., 1.], args=(grid[i][0], grid[i][1], M, sigma, q))   #Integral em x
    result_y[i] = quadva(no_alpha_y, [0., 1.], args=(grid[i][0], grid[i][1], M, sigma, q))   #Integral em y


25.1 s ± 39.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
import time

1614369459.2632694

In [27]:
start = time.time()
for i in range(len(grid)):                      #Começo do loop
    result_x[i] = quadva(no_alpha_x, [0., 1.], args=(grid[i][0], grid[i][1], M, sigma, q))   #Integral em x
    result_y[i] = quadva(no_alpha_y, [0., 1.], args=(grid[i][0], grid[i][1], M, sigma, q))   #Integral em y
    
print(time.time()-start)

25.170894384384155


In [30]:
print("Ellapsed time:",time.time()-start)

Ellapsed time: 589.0055205821991
